In [1]:
# General imports
import numpy as np
import pandas as pd 
import os
from IPython.display import clear_output

#Data Import and Timing
import pandas_datareader.data as dr
import datetime 

#Graphing
import matplotlib.pyplot as plt 

#Setting up model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score
from sklearn.preprocessing import MinMaxScaler,RobustScaler

#hyperparameter tuning 
from bayes_opt import BayesianOptimization
import shap

#Learners
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso,Ridge,RidgeCV,LinearRegression
import lightgbm as lgb

#feature calculation
import ta

import featuretools as ft

from tsfresh import extract_relevant_features
from tsfresh.feature_extraction import feature_calculators

import scipy as sp

from Stock_Utility_functions import *

def BestStackFinder(**weights):
    #"best compilation determiner"
    global for_weight_preds
    import pandas as pd
    preds=for_weight_preds
    weight_list = []

    ## returns vars to use for multiplication

    for key, value in weights.items():
        weight_list.append(value)

    ## Multiply stacking weights by predictions and add them up
    comb_preds = np.zeros(preds.shape[0])
    Y_test = preds.iloc[:,0]
    preds = preds.iloc[:,1:]
    for weight,col in zip( weight_list, preds.columns.values):
        comb_preds1 = weight*np.asarray(preds[col])
        comb_preds = comb_preds +comb_preds1 

    Y_Test_Categorical = np.where(Y_test.values <= 0, 0, 1)
    Preds_Categorical = np.where(comb_preds <= 0, 0, 1)
    return -1*(mean_absolute_error(Y_test,comb_preds))
clear_output()

ModuleNotFoundError: No module named 'bayes_opt'

In [ ]:
records=process()

In [ ]:
ticker_df = pd.DataFrame(records,columns=['Tickers',"Company_Name","Industry"])

In [ ]:
from gplearn.functions import make_function
from gplearn.genetic import SymbolicRegressor

def th(x):
    return np.tanh(x)

gptanh = make_function(th, 'tanh', 1)
function_set = ['add', 'sub', 'mul', 'div', 'inv', 'abs', 'neg', 'max', 'min', gptanh]

model = SymbolicRegressor(metric = 'mean absolute error',stopping_criteria=20,n_jobs = -1)

# estimator.fit(XX_train,YY_train)
# print(estimator.score(XX_test, YY_test,deep=True))

# estimator.fit(X_train,Y_train)
# genetic_preds = estimator.predict(X_test)

In [ ]:
#Webscraping
from bs4 import BeautifulSoup
import csv
from os import mkdir
from os.path import exists, join
import urllib.request as request

In [ ]:
LGB_PARAMS = {'num_leaves': 200,
            'boosting_type': 'dart',
             'min_data_in_leaf': 30, 
             'learning_rate': 0.02,
             "min_child_samples": 2,
             "feature_fraction": 0.9,
             "bagging_freq": 5,
             "bagging_fraction": 0.9 ,
             "bagging_seed": 11,
             "metric": 'mae',
             "lambda_l1": 0.1,
             "verbosity": -1,
             "nthread": 4,
             "random_state": 4590}

In [ ]:
ticker_df

In [ ]:
train_start_date = '2017-10-02'
train_stop_date  = '2019-10-17'
shift_window_of_days = 1


Tommorow_Predictions = []
ticker_for_number_features = 'EBAY'
Official_Train_Set,target = gen_feats(ticker_for_number_features,train_start_date,train_stop_date,shift_window_of_days)
number_of_features = len(Official_Train_Set.columns)

for Industry in list(ticker_df['Industry'].unique())[6:8]:
    print('----------------------------'+Industry+'------------------------------')
    tickers = list(ticker_df.loc[ticker_df['Industry'] == Industry,'Tickers'])
    
    run_over = tickers[:6]
    # Repeat the process happening in the loop below, but with every one of the previous stocks
    for ticker in run_over:
        
        for_weight_preds = pd.DataFrame()
        
        final_preds = pd.DataFrame()
        preds = pd.DataFrame()
        
        Master_DataFrame = pd.DataFrame()
        Targets_DataFrame = pd.DataFrame()
        
        tickers_for_loop = run_over
        
        Official_Train_Set,target = gen_feats(ticker,train_start_date,train_stop_date,shift_window_of_days)
        Official_Train_Set_1, Official_Train_Set_2, target_1, target_2 = train_test_split(Official_Train_Set,target,
                                                                test_size =0.5,shuffle = False)

        tickers_for_loop.remove(ticker)
        for_weight_preds['target'] = target_1.values.flatten()
        #Every other stock is trained on, and then, the booster predicts on the holdout stock. results are weighted.
        for iteration_number,ticker1 in enumerate(tickers_for_loop):
            
            X_T,Y_T = gen_feats(ticker1,train_start_date,train_stop_date,shift_window_of_days)
            
            Master_DataFrame = pd.concat((Master_DataFrame,X_T),axis=1)
            Targets_DataFrame = pd.concat((Targets_DataFrame,Y_T),axis=1)
            
            X_train = Master_DataFrame.loc[:,[x for x in list(Master_DataFrame.columns) if ticker1 in x]]
            Y_train = Targets_DataFrame[ticker1]
#             model.fit(X_train, Y_train)
            Train = lgb.Dataset(X_train, label=Y_train)

            model = lgb.train(LGB_PARAMS,Train,num_boost_round =10)
            
            for_weight_preds['prediction'+str(iteration_number)] = model.predict(Official_Train_Set_1)
            final_preds['prediction'+str(iteration_number)] = model.predict(Official_Train_Set_2)
            
        weights = ReturnBestStackingWeights(for_weight_preds,n_iter=2)
        stacked_predictions = gen_stack_preds(final_preds,weights)    
        
        print('The Error  for '+ticker+' is '+str(mean_absolute_error(target_2,stacked_predictions)))

    #      Find how accurate categorically
        Y_Test_Categorical = np.where(target_2.values <= 0, 0, 1)
        Preds_Categorical = np.where(stacked_predictions <= 0, 0, 1)
        
        print('Categorical Accuracy for '+ticker+ ' is '+str(accuracy_score(Y_Test_Categorical,Preds_Categorical)))
#         Tommorow_Predictions.append([ticker,list(stacked_predictions)[-1]])
#         if accuracy_score(Y_Test_Categorical,Preds_Categorical) >= 0.7:
#             plt.figure(figsize=(30,30))
#             plt.plot((np.abs(target_2.values[-100:])*np.where(target_2.values[-100:] == 0, -1, 1)))
#             plt.plot(np.abs(stacked_predictions[-100:])*np.where(stacked_predictions[-100:] == 0, -1, 1))
#             plt.show()
#     print('-------------------------------------------------done!-----------------------------------------------------')

In [ ]:
# Find drops and jumps 
# The criteria is on average change (std)
# check to see if stock usually grows or drops, then check the current state in last 2-3 months (3 is used in case of more fluctiations, 
# eg. more drops or growth, this will help with bigger picture )
# in cases of growth 